In [ ]:
# NEW - STACKSTAC call is in LatLon 4326 #########################
# convert point latlons to image crs and create buffer using meters
stac_crs = stac_item.properties["proj:epsg"]
proj_latlon_to_stac = pyproj.Transformer.from_crs(
    4326, stac_crs, always_xy=True
)
x_utm, y_utm = proj_latlon_to_stac.transform(lon, lat)
x_min, x_max = x_utm - self.buffer, x_utm + self.buffer
y_min, y_max = y_utm - self.buffer, y_utm + self.buffer

# convert buffer bounds back to latlon 4326
proj_stac_to_latlon = pyproj.Transformer.from_crs(
    stac_crs, 4326, always_xy=True
)
x_min, y_min = proj_stac_to_latlon.transform(x_min, y_min)
x_max, y_max = proj_stac_to_latlon.transform(x_max, y_max)

# do stackstac call in 4326. Rescale must be True (default).
# NOTE: rough converted resolution... How to get this to be exact??
rough_latlon_resolution = (1 / 111111) * self.resolution
xarray = stackstac.stack(
    stac_item,
    assets=self.bands,
    epsg=4326,
    resolution=rough_latlon_resolution,
    bounds_latlon=[x_min, y_min, x_max, y_max],
    dtype=np.uint8,
    fill_value=0,
    # rescale=False,
    # snap_bounds=False
)
##################################################################

In [ ]:
# import argparse

# def parse_args():
#     """Parses arguments for the script."""

#     parser = argparse.ArgumentParser()
#     parser.add_argument(
#         "--admin-level",
#         action="store",
#         default="village",
#         help="Adminstatrative level to use for the shapefiles. Options: 'village', 'subdistrict', 'district', or 'state.",
#     )

#     args = parser.parse_args()

#     return args

def main():
    """Preprocess SHRUG keys (rural and urban) by adding shapes and saving to file."""

    # args = parse_args()
    # level = args.admin_level

In [ ]:
# # select list of columns to drop from the data so only features and target are left
# shrug_key_cols = [
#     "shrid",
#     "pc11_s_id",
#     "pc11_d_id",
#     "pc11_sd_id",
#     "pc11_tv_id",
#     "tv_name",
#     "geometry",
#     "geometry_village"
#     "geometry_subdistrict",
# ]
# geo_cols = ["Lat", "Lon"] + shrug_key_cols

# shrug_secc_cols = [
#     "shrid",
#     "secc_inc_cultiv_share",
#     "nco2d_cultiv_share",
#     "secc_cons_pc_rural",
#     "secc_cons_pc_urban",
#     "secc_pov_rate_rural",
#     "secc_pov_rate_urban",
#     "secc_pov_rate_tend_rural",
#     "secc_pov_rate_tend_urban",
#     "num_members_mean_rural",
#     "num_members_mean_urban",
# ]
# shrug_secc_cols.remove(y_name)

# cols_to_drop = shrug_secc_cols + geo_cols

# X = gdf_clean.drop(cols_to_drop + [y_name], axis=1)

## Visualize NaNs

In [ ]:
notnull = ~(mosaiks_features["1000"].isnull())*1

f, ax = plt.subplots(1,1)
notnull.cumsum().plot(ax=ax)
ax.plot([0,96167],[0,96167],c="r")
plt.show()

## Load temp files, combine, and save

In [ ]:
import glob

filepaths = sorted(glob.glob(mosaiks_path+"2021_*.parquet.gzip"))

filepaths

temp = pd.read_parquet(
    "/home/jovyan/ds_nudge_up/data/01_preprocessed/mosaiks_features/2021_3km_v4000_L8_60000__0_to_36166.parquet.gzip"
)
temp.index = range(60000, 96167)
temp.to_parquet(
    "/home/jovyan/ds_nudge_up/data/01_preprocessed/mosaiks_features/2021_3km_v4000_L8_60000_to_96166.parquet.gzip", 
    compression="gzip"
)

In [ ]:
filename = '2021_3km_v4000_L8_0_to_96166.parquet.gzip'

l_dfs = []
for filepath in filepaths:
    l_dfs.append(pd.read_parquet(filepath))

mosaiks_features = pd.concat(l_dfs, axis=0)

mosaiks_features.to_parquet(mosaiks_path+filename, compression="gzip")

In [ ]:
# filter points with buffer

states = gpd.read_file(f"{DATA_ROOT}/00_raw/SHRUG/geometries_shrug-v1.5.samosa-open-polygons-shp/state.shp")
states['zero_column'] = 0
country = states.dissolve(by='zero_column')
BUFFER_DISTANCE = 0.005


def filter_points_with_buffer(points_gdf, shape, buffer_distance):
    # This buffer ensures that no points are take at the border
    # which would lead to duplication with neighboring countries

    return points_gdf[points_gdf.within(shape.unary_union.buffer(buffer_distance))]

points_gdf = filter_points_with_buffer(points_gdf, country, BUFFER_DISTANCE)

## Load MOSAIKS features

In [ ]:
# Load custom-made mosaiks features
# data_label = "3km_v1024_L8"
# filename = 'mosaiks_2013_L8_0.015b_all_points.parquet.gzip'

# data_label = "1km_v1024_L8"
# filename = 'mosaiks_2013_L8_0.005b_all_points.parquet.gzip'

# lat_name, lon_name = "lat", "lon"

# Load data
# mosaiks_features = pd.read_parquet(mosaiks_path+filename)

In [ ]:
# # OR using old data downloaded from the online API
# data_label = "old_v4000"
# filename = "mosaiks_downloaded_old_features_rounded.csv"
# lat_name, lon_name = "Lat", "Lon"

# # Load data
# mosaiks_features = pd.read_csv(mosaiks_path+filename)

In [ ]:
mosaiks_features_gdf = load_mosaiks_data("INDIA_SHRUG_Mosaiks_features.csv")
# mosaiks_features_gdf.head()

In [ ]:
mosaiks_features_gdf.shape

## Load preprocessed SHRUG keys with shapes

In [ ]:
shrug_key_geoms = load_gdf("01_preprocessed/SHRUG/shrug_pc11r_key_with_shapes", "shrug_pc11r_key_with_shapes.shp")
# shrug_key_geoms.head()

## Use SHRUG shapes to add `shrid`s to MOSAIKS features

In [ ]:
mosaiks_features_gdf = add_shrid_to_mosaiks(mosaiks_features_gdf, shrug_key_geoms)

## Import SHRUG SECC (target)

In [ ]:
file_path = Path.cwd().parent / "data" / "00_raw" / "SHRUG" / "shrug-v1.5.samosa-secc-csv" / "shrug_secc.csv"
shrug_secc = pd.read_csv(file_path)
# shrug_secc.head()

In [ ]:
shrug_secc.shape

## Match target to features using `shrid`s

In [ ]:
df = merge_mosaiks_and_secc(mosaiks_features_gdf, shrug_secc)
# df.head()

In [ ]:
df.shape